# Road Follower - Train Model（训练模型）
在这个笔记本中，我们将训练一个神经网络来获取一个输入图像，并输出一组对应于目标的x，y值。
我们将使用PyTorch深度学习框架来训练ResNet18神经网络架构模型，以供道路跟随者应用。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### 下载和提取数据
在开始之前，您应该上传您在robot上的 ``data-collection.ipynb``笔记本中创建的``road-following.zip``文件。
> 如果要在 Jetbot 训练收集数据，可以跳过这个！

然后，您应该通过调用下面的命令来提取此数据集：

In [ ]:
!unzip -q road_following.zip

您应该会在文件浏览器中看到一个名为“dataset_all”的文件夹。

### 创建数据集实例
在这里，我们创建了一个自定义的 ``torch.utils.data.Dataset``实现，它执行``__len__`` 和``__getitem__`` 函数。这个类负责从图像文件名加载图像并解析x，y值。因为我们实现了 ``torch.utils.data.Dataset``类，我们可以使用所有torch数据实用程序：）

我们硬编码一些转换（如颜色抖动）到我们的数据集。我们做了随机的水平翻转（如果你想沿着一条非对称的道路，比如一条路。我们需要“保持正确”。如果小车是否遵循某种约定并不重要，那么您可以启用翻转来扩充数据集。

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### 将数据集拆分为训练集和测试集
一旦我们读取了数据集，我们将在火车和测试集中分割数据集。在这个例子中，将训练与测试数据集分开为90%-10%两部分，测试数据集将用于验证我们训练的模型的准确性。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### 创建数据加载程序以批量加载数据

我们使用 ``DataLoader`` 类来批量加载数据、洗牌数据并允许使用多个子进程。在本例中，我们使用16的batch size。批处理大小将基于GPU可用的内存，它会影响模型的准确性。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### 定义神经网络模型
我们在Pythorch TorchVision上使用ResNet-18模型。在一个称为迁移学习（transfer learning）过程中，我们可以重新利用一个预先训练好的模型（在数百万张图片上训练）来完成一个新的任务，这个任务的可用数据可能要少得多。
有关ResNet-18的更多详细信息：https://github.com/pytorch/vision/blob/master/torchvision/models/ResNet.py
有关转移学习的更多详细信息：https://www.youtube.com/watch？v=yofjFQddwHE

In [ ]:
model = models.resnet18(pretrained=True)

ResNet模型有完全连接（fc）最后一层的512作为 ``in-features``，我们以 ``out-features``为1进行回归训练

最后，我们将模型转移到GPU上执行

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### 训练回归模型：
我们训练了50个阶段，如果减少损失，就可以保存最好的模型。

In [ ]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

一旦模型经过训练，它将生成 ``best_steering_model_xy.pth`` 文件，您可以在现场演示笔记本中使用该文件进行推理。

如果您在JetBot以外的其他机器上进行训练，则需要将此文件上传到JetBot的 ``road-following`` 示例文件夹中。